In [ ]:
'''高德爬店铺'''
import requests
import json

def get_pois(location, radius, types):
    key = "从高德平台申请的key"
    url = 'https://restapi.amap.com/v5/place/around'
    params = {
        'key': key,
        'location': location,
        'radius': radius,
        'types': types,
        'offset': 1,
        'page': 1,  
        'extensions': 'all' 
    }
    all_pois = []
    while True:
        response = requests.get(url, params=params)
        data = response.json()

        # 如果请求失败，结束循环
        if data['status'] != '1':
            break
        pois = data.get('pois', [])

        # 如果没有POI数据，结束循环
        if not pois:
            break
        all_pois.extend(pois)

        # 如果POI数量小于offset，说明没有更多数据，结束循环
        if len(pois) < params['offset']:
            break
        params['page'] += 1

    return all_pois

def save_poi_details(pois, filename='shop_details.json'):
    print(f"保存POI详细信息到文件: {filename}")
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(pois, f, ensure_ascii=False, indent=4)
    print("保存完成。")

if __name__ == '__main__':
    location = '114.290150,30.586953'
    radius = '1000' 
    types_list = ['050000', '050400', '050500', '060000', '070000', '080000', '090000', '110000', '140000', '200000', '220000', '990000', '991000']

    for types in types_list:

        print(f"开始获取POI信息，位置: {location}, 半径: {radius}, 类型: {types}")
        pois = get_pois(location, radius, types)

        # 保存POI详细信息到文件
        save_file_name = fr"D:\Desktop\site_around\{types}_list.json"
        save_poi_details(pois, save_file_name)

        # 打印POI名称
        shop_names = [poi['name'] for poi in pois]
        for name in shop_names:
            print(f"店铺名称: {name}")
        print(f'总共找到 {len(shop_names)} 个店铺')

In [ ]:
'''搜中心点附近的osm'''
import os
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
from requests.exceptions import RequestException
from warnings import simplefilter

# 忽略警告信息
simplefilter(action='ignore', category=FutureWarning)


# 设置中心点的经纬度和搜索半径（单位：米）    location = '114.290150,30.586953'
latitude = 30.586953  # 示例纬度
longitude = 114.290150 # 示例经度
distance = 2000  # 半径，单位为米

# 定义输出文件夹路径
output_folder = "D:/Desktop/gaode/osmnx_data/"

# 检查输出文件夹是否存在，如果不存在则创建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 定义一个保存函数，处理字段名和可能的保存错误
def save_shapefile(data, filename):
    try:
        # 去除重复列名并截断字段名
        data = data.loc[:, ~data.columns.duplicated()]
        data.columns = [col[:10] for col in data.columns]
        data.to_file(filename, driver="ESRI Shapefile")
        print(f"成功保存文件: {filename}")
    except Exception as e:
        print(f"保存文件失败: {filename}, 错误: {e}")

# 定义一个请求数据的函数，包含重试机制
def fetch_data(func, *args, **kwargs):
    retries = 3
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"请求失败，重试 {attempt+1}/{retries} 次...")
            if attempt == retries - 1:
                print(f"请求多次失败，错误: {e}")
                return None

# 1. 获取道路网格信息
road_network = ox.graph_from_point((latitude, longitude), dist=distance, network_type='drive')
road_edges = ox.graph_to_gdfs(road_network, nodes=False, edges=True)
save_shapefile(road_edges, os.path.join(output_folder, "road_network.shp"))

# 2. 获取建筑物信息
buildings = fetch_data(ox.features_from_point, (latitude, longitude), {'building': True}, distance)
if buildings is not None:
    save_shapefile(buildings, os.path.join(output_folder, "buildings.shp"))

# 3. 获取绿地信息（如公园、花园等）
green_areas = fetch_data(ox.features_from_point, (latitude, longitude), {'leisure': 'park'}, distance)
if green_areas is not None:
    save_shapefile(green_areas, os.path.join(output_folder, "green_areas.shp"))

# 4. 获取水体信息（如河流、湖泊等）
water_bodies = fetch_data(ox.features_from_point, (latitude, longitude), {'natural': 'water'}, distance)
if water_bodies is not None:
    save_shapefile(water_bodies, os.path.join(output_folder, "water_bodies.shp"))

# 5. 获取交通设施信息（如公交车站、停车场等）
transport_facilities = fetch_data(ox.features_from_point, (latitude, longitude), {'amenity': 'bus_station'}, distance)
if transport_facilities is not None:
    save_shapefile(transport_facilities, os.path.join(output_folder, "transport_facilities.shp"))

# 6. 获取土地用途信息（如住宅区、商业区等）
land_use = fetch_data(ox.features_from_point, (latitude, longitude), {'landuse': ['residential', 'commercial']}, distance)
if land_use is not None:
    save_shapefile(land_use, os.path.join(output_folder, "land_use.shp"))

# 7. 获取兴趣点（POI）信息（如餐馆、商店、医院等）
poi = fetch_data(ox.features_from_point, (latitude, longitude), {'amenity': ['restaurant', 'hospital', 'school']}, distance)
if poi is not None:
    save_shapefile(poi, os.path.join(output_folder, "poi.shp"))

print("所有数据已尝试保存为 Shapefile 格式，检查输出以确认结果。")
